## Step-2

In [ ]:
# Pydicom used for dcm image resize for Inference
!conda install '/kaggle/input/pydicom-conda-helper/libjpeg-turbo-2.1.0-h7f98852_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/libgcc-ng-9.3.0-h2828fa1_19.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/gdcm-2.8.9-py37h500ead1_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/conda-4.10.1-py37h89c1867_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/certifi-2020.12.5-py37h89c1867_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/openssl-1.1.1k-h7f98852_0.tar.bz2' -c conda-forge -y

In [ ]:
# !pip install -U fastai --upgrade
!pip install -q git+https://github.com/rwightman/pytorch-image-models.git
!pip install -q iterative-stratification
!pip install -q wandb --upgrade
!pip install -q nbdev

!pip install timm

In [ ]:
import os

device = 'CPU'
if 'TPU_NAME' in os.environ.keys():
    if os.environ['XRT_TPU_CONFIG'] is not None: device = 'TPU'
elif 'CUDA_VERSION' in os.environ.keys():
    if os.environ['CUDA_VERSION'] is not None: device = 'GPU'

print(device)

In [ ]:
import pandas as pd
import timm
from timm import *

from fastai.vision.all import *
from fastai.vision.learner import _update_first_layer
from fastai.callback.wandb import *

In [ ]:
path_dcm = Path('../input/siim-covid19-detection')
path_image_resize = '/kaggle/input/siim-covid19-512-images-and-metadata/train'
path_model_save = Path('/kaggle/working/models')

## Prepare Label Data
2Class Label Data from Source  
-2Class is really just positive and none/negative; issue with less then 3 classes, so created two positive classes

In [ ]:
df_train_2cls = pd.read_csv(path_dcm / 'train_image_level.csv')
df_train_2cls['none'] = 0
df_train_2cls['positiveA'] = 0
df_train_2cls['positiveB'] = 0

In [ ]:
for i in range(df_train_2cls.shape[0]):
    if df_train_2cls.loc[i,'label'] == 'none 1 0 0 1 1':
        df_train_2cls.loc[i,'none'] = 1
    else:
        if (i % 2) == 0:
           df_train_2cls.loc[i,'positiveA'] = 1
        else:
           df_train_2cls.loc[i,'positiveB'] = 1

In [ ]:
# df_train_2cls.to_csv('train.csv',index=False)
df_train_2cls.head()

In [ ]:
label_cols = df_train_2cls.columns[4]
label_cols

df_annotations = df_train_2cls.copy()

In [ ]:
# df_study_lvl = pd.read_csv(path_dcm / "train_study_level.csv")
# df_study_lvl.rename({'id':'study_id',
#                       'Negative for Pneumonia':'negative',
#                       'Typical Appearance':'typical',
#                       'Indeterminate Appearance':'indeterminate',
#                       'Atypical Appearance':'atypical'}, axis=1, inplace=True)

# df_image_lvl = pd.read_csv('/kaggle/input/siim-covid19-detection/train_image_level.csv')
# df_image_lvl['study_id'] = df_image_lvl['StudyInstanceUID'].apply(lambda idx: idx+"_study")

# df_annotations = df_image_lvl.merge(df_study_lvl, on='study_id', how='outer')
# df_annotations.head(3)

In [ ]:
df_annotations['image_path'] = df_annotations['id'].map(lambda x:os.path.join(path_image_resize,
                                                                              str(x)+'.png'))
df_annotations.head()

In [ ]:
label_names = ['negative','typical','indeterminate','atypical']
label_names = ['none','positiveA','positiveB']

def get_labels(row):
    labels_str = ''
    for key in label_names:
        if row[key]==1:
             labels_str = labels_str+' '+key if labels_str else key
    return labels_str

df_annotations['labels'] = df_annotations[label_names].apply(get_labels, axis=1)

df_annotations.head()

## Training Global Config

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    
    if device=='GPU':
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True

In [ ]:
class Config:
    seed_val = 111
    seed_everything(seed_val)
    fold_num = 0
    job = 1
    num_classes = 3
    input_dims = 512
    model_arch = "efficientnetv2_rw_s"
    batch_size = 32
    num_workers = 0
    kfold = 5
    loss_func = CrossEntropyLossFlat() # or LabelSmoothingCrossEntropyFlat()
    metrics = [error_rate, accuracy, RocAuc(average='macro'), F1Score(average='macro')]
    job_name = f'{model_arch}_ImageClass_2cls_fold{fold_num}_job{job}'
    print("Job Name:", job_name)

    wandb_project = 'SIIM_classifier_public'
    wandb_run_name = job_name
    
    if device=='GPU':
        fp16 = True
    else:
        fp16 = False
    
cfg = Config()

In [ ]:
# Converting global config class object to a dictionary to log using Wandb

config_dict = dict(vars(Config))
config_dict = {k:(v if type(v)==int else str(v)) for (k,v) in config_dict.items() if '__' not in k}
config_dict

In [ ]:
print('Unique labels:', df_annotations.labels.unique())

In [ ]:
# Set-up KFold split
from sklearn.model_selection import GroupKFold, train_test_split

df_annotations['fold'] = -1
grp_kfold  = GroupKFold(n_splits = cfg.kfold)
for fold, (train_index, val_index) in enumerate(grp_kfold.split(df_annotations,
                                                              groups=df_annotations.StudyInstanceUID.tolist())):
    df_annotations.loc[val_index, 'fold'] = fold
df_annotations.sample(3)

## Augmentation & Dataloading

In [ ]:
import albumentations as A

# Source: https://forums.fast.ai/t/albumentation-transformations-for-train-and-test-dataset/82642
class AlbumentationsTransform(RandTransform):
    split_idx,order=None,2
    def __init__(self, train_aug, valid_aug): store_attr()
    
    def before_call(self, b, split_idx):
        self.idx = split_idx
    
    def encodes(self, img: PILImage):
        if self.idx == 0:
            aug_img = self.train_aug(image=np.array(img))['image']
        else:
            aug_img = self.valid_aug(image=np.array(img))['image']
        return PILImage.create(aug_img)
    
def get_train_aug(): 
    return A.Compose([
#         A.RandomResizedCrop(cfg.input_dims,cfg.input_dims), 
        A.Resize(cfg.input_dims, cfg.input_dims, p=1.0),
        A.HorizontalFlip(p=0.5),
        A.ShiftScaleRotate(shift_limit=0.02, scale_limit=0.1, rotate_limit=10, p=0.5),
        A.IAAPerspective(scale=(0.02, 0.04), p=0.5),
        A.RandomBrightnessContrast(0.1, 0.1, p=0.5),
        A.OneOf([A.CLAHE(),
                 A.HueSaturationValue(0.2, 0.2, 0.2, p=0.5)
                ],p=0.4),
        A.OneOf([A.CoarseDropout(),
                 A.Cutout()], p=0.5)
    ])

def get_valid_aug():
    return A.Compose([A.Resize(cfg.input_dims, cfg.input_dims, p=1.0)], p=1.0)

## Batch tfms on gpu --> so faster
## Only fastai has it so make use of these transformations as much as possible
## Check augtransforms
## item tfms on cpu and happens to one image at a time
## RandomResizedCrop behaves differently with train and val transforms (tries to get the biggeset centre crop from an image)

item_tfms = AlbumentationsTransform(get_train_aug(), get_valid_aug())
batch_tfms = [Normalize.from_stats(*imagenet_stats)]

In [ ]:
## Explain different dataset and data loading mechanisms

val_indices = list(df_annotations[df_annotations['fold'] == cfg.fold_num].image_path.unique())

data_block = DataBlock(blocks=(ImageBlock, CategoryBlock),
                   splitter=MaskSplitter(list(df_annotations['fold'] == fold)),
                   get_x=ColReader('image_path'),
                   get_y=ColReader('labels'),
                   item_tfms=item_tfms,
                   batch_tfms=batch_tfms)

dls = data_block.dataloaders(df_annotations,
                            bs=cfg.batch_size,
                            num_workers=cfg.num_workers)

dls.show_batch(figsize=(18,15), max_n=8, nrows=2)

In [ ]:
print("Class2Index Mapping:")
dls.vocab.o2i

## Set-up TIMM Learner

In [ ]:
# Adapted from https://walkwithfastai.com/vision.external.timm

from fastai.vision.learner import _add_norm

def create_timm_body(arch:str, pretrained=True, cut=None, n_in=3):
    "Creates a body from any model in the `timm` library."
    model = create_model(arch, pretrained=pretrained, num_classes=0, global_pool='')
    _update_first_layer(model, n_in, pretrained)
    if cut is None:
        ll = list(enumerate(model.children()))
        cut = next(i for i,o in reversed(ll) if has_pool_type(o))
    if isinstance(cut, int): return nn.Sequential(*list(model.children())[:cut])
    elif callable(cut): return cut(model)
    else: raise NamedError("cut must be either integer or function")

def create_timm_model(arch:str, n_out, cut=None, pretrained=True, n_in=3,
                      init=nn.init.kaiming_normal_, custom_head=None,
                      concat_pool=True, **kwargs):
    "Create custom architecture using `arch`, `n_in` and `n_out` from the `timm` library"
    body = create_timm_body(arch, pretrained, None, n_in)
    if custom_head is None:
        nf = num_features_model(nn.Sequential(*body.children()))
        head = create_head(nf, n_out, concat_pool=concat_pool, **kwargs)
    else: head = custom_head
    model = nn.Sequential(body, head)
    if init is not None: apply_init(model[1], init)
    return model

def timm_learner(dls, arch:str, loss_func=None, pretrained=True, cut=None, splitter=None,
                y_range=None, config=None, n_out=None, normalize=True, fp16=False, **kwargs):
    "Build a convnet style learner from `dls` and `arch` using the `timm` library"
    if config is None: config = {}
    if n_out is None: n_out = get_c(dls)
    assert n_out, "`n_out` is not defined, and could not be inferred from data, set `dls.c` or pass `n_out`"
    if y_range is None and 'y_range' in config: y_range = config.pop('y_range')
    model = create_timm_model(arch, n_out, default_split, pretrained, y_range=y_range, **config)
    learn = Learner(dls, model, loss_func=loss_func, splitter=default_split, **kwargs)
    if pretrained: learn.freeze()
    
    # Enable Mixed Precision Training
    if fp16: learn.to_non_native_fp16()
#     if fp16: learn.to_fp16(growth_factor=1.0)
    return learn

In [ ]:
cbs = [
#     WandbCallback(log='gradients',
#                   log_preds=True,
#                   log_model=True,
#                   log_dataset=False,
#                   dataset_name=None,
#                   valid_dl=None,
#                   n_preds=36,
#                   seed=cfg.seed_val,
#                   reorder=True),   
    SaveModelCallback(monitor='valid_loss',
                      comp=None,
                      min_delta=0.0,
                      fname=cfg.job_name,
                      every_epoch=False,
                      with_opt=False,
                      reset_on_fit=True)
      ]

In [ ]:
learn = timm_learner(dls,
                     cfg.model_arch,
                     loss_func=cfg.loss_func,
                     pretrained=True,
                     opt_func=ranger,
#                      splitter=default_split,
                     fp16=cfg.fp16,
                     metrics=cfg.metrics,
                     cbs=cbs)
# learn.summary()

In [ ]:
frozen_params = filter(lambda p: not p.requires_grad, learn.model.parameters())
unfrozen_params = filter(lambda p: p.requires_grad, learn.model.parameters())

print(f'Total Parameters: {sum([np.prod(p.size()) for p in learn.model.parameters()])}')
print(f'Frozen Parameters: {sum([np.prod(p.size()) for p in frozen_params])}')
print(f'Unfrozen Parameters: {sum([np.prod(p.size()) for p in unfrozen_params])}')

## Training  
### Step-1

In [ ]:
# learn.fit_one_cycle(10, 5e-3)
learn.fit_one_cycle(3, 5e-3)

### Step-2

In [ ]:
from fastai.callback.tracker import SaveModelCallback, CancelStepException
# model_save_name = path_model_save / cfg.job_name
model_save_name = path_model_save / 'step2-ImageClass-2cl'
sm = SaveModelCallback(fname=str(model_save_name))

In [ ]:
learn.unfreeze()
# learn.fit_one_cycle(3, lr_max=slice(1e-7, 5e-5), cbs=sm)
learn.fit_one_cycle(1, lr_max=slice(1e-7, 5e-5), cbs=sm)

## Confusion Matrix

In [ ]:
# learn.load(file=cfg.job_name)

# interp = ClassificationInterpretation.from_learner(learn)
# interp.plot_confusion_matrix(figsize=(8,8), dpi=60)